## NHL API 

### Description

The following code queries the NHL database fore team ids and team season stats.

In [1]:
# Import libraries
import requests
import pandas as pd
import time

### Team IDs

In [2]:
# Function to retrieve NHL team ids from NHL API
def nhl_id(): 

  # User Guide : https://www.kevinsidwar.com/iot/2017/7/1/the-undocumented-nhl-stats-api

    def nhl(team_id):
        url = "https://statsapi.web.nhl.com/api/v1/teams"

        response = requests.get(url)
        data = response.json()
        data = data['teams']
        data = data[team_id]
        data_id = data['id']
        data_name = data['name']

        n = [{
            
            "id" : data_id,
            "name" : data_name,
        }]

        return n

    index = 0
    request = 0
    max_request = 34
    n = []

    while request < max_request:

        try:
            
            x = nhl(index)
            n = n + x
            index += 1
            request += 1

        except IndexError:

            df = pd.DataFrame(n)

            return df

In [3]:
# Run team id function
nhl_id()

,id,name
0,1,New Jersey Devils
1,2,New York Islanders
2,3,New York Rangers
3,4,Philadelphia Flyers
4,5,Pittsburgh Penguins
5,6,Boston Bruins
6,7,Buffalo Sabres
7,8,Montréal Canadiens
8,9,Ottawa Senators
9,10,Toronto Maple Leafs


The NHL team IDs are collected to get there individual team season stats.

### Season Stats

In [17]:
# Function to query team stats for a single given season
def nhl(year):

    team_id = nhl_id()

    def nhl_stats(id_num, year_range):

        url = "http://statsapi.web.nhl.com/api/v1/teams/{}?expand=team.stats&season={}".format(id_num, year_range)

        response = requests.get(url)
        data = response.json()
        data = data['teams']
        data = data[0]
        data = data['teamStats']
        data = data[0]
        data = data['splits']
        data = data[0]
        data = data['stat']

        return data
  
    x = -1
    request = -1
    max_request = 30
    n = []

    while request < max_request:

        try:

            x += 1
            id_num = team_id.loc[x, 'id']
            id_name = team_id.loc[x, 'name']
            request += 1
            stats = nhl_stats(id_num, year)
            stats['id'] = id_num
            stats['name'] = id_name
            stats['year_range'] = year
            n = n + [stats]
            time.sleep(1)

        except:

            continue
   
    return n

# Function to loop through, using the above function, for 35 seasons of team stats
def nhl_table(year1_range, max_year):

    year1 = int(year1_range[0:4])
    year2 = int(year1_range[5:9])
    request = -1
    max_request = (max_year - year2) - 1
    index = -1
    n = []

    while request < max_request:

        index += 1
        year1 += 1
        year2 += 1
        year_range = '{}{}'.format(year1, year2)
        request += 1
        stats = nhl(year = year_range)
        n = n + stats
        time.sleep(1)

    n = pd.DataFrame(n)

    return n

### 1983 - 2020 Team Stats

In [18]:
# season stats data frame
nhl_df = nhl_table(year1_range = '1982,1983', max_year = 2020)

print('\n')
nhl_df.info()
nhl_df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 31 columns):
gamesPlayed               895 non-null int64
wins                      895 non-null int64
losses                    895 non-null int64
ot                        895 non-null int64
pts                       895 non-null int64
ptPctg                    895 non-null object
goalsPerGame              895 non-null float64
goalsAgainstPerGame       895 non-null float64
evGGARatio                895 non-null float64
powerPlayPercentage       895 non-null object
powerPlayGoals            895 non-null float64
powerPlayGoalsAgainst     895 non-null float64
powerPlayOpportunities    895 non-null float64
penaltyKillPercentage     895 non-null object
shotsPerGame              895 non-null float64
shotsAllowed              895 non-null float64
winScoreFirst             895 non-null float64
winOppScoreFirst          895 non-null float64
winLeadFirstPer           895 non-null float64
winLeadSecond

,gamesPlayed,wins,losses,ot,pts,ptPctg,goalsPerGame,goalsAgainstPerGame,evGGARatio,powerPlayPercentage,...,winOutshotByOpp,faceOffsTaken,faceOffsWon,faceOffsLost,faceOffWinPercentage,shootingPctg,savePctg,id,name,year_range
0,80,17,56,0,41,25.6,2.888,4.375,0.6780,14.3,...,0.188,0.0,0.0,0.0,0.0,10.9,0.863,1,New Jersey Devils,19831984
1,80,50,26,0,104,65.0,4.463,3.363,1.3722,22.5,...,0.532,0.0,0.0,0.0,0.0,14.7,0.899,2,New York Islanders,19831984
2,80,42,29,0,93,58.1,3.925,3.800,1.0392,25.1,...,0.467,0.0,0.0,0.0,0.0,12.3,0.880,3,New York Rangers,19831984
3,80,44,26,0,98,61.3,4.375,3.625,1.3646,20.2,...,0.286,0.0,0.0,0.0,0.0,11.9,0.873,4,Philadelphia Flyers,19831984
4,80,16,58,0,38,23.8,3.175,4.875,0.6466,20.9,...,0.183,0.0,0.0,0.0,0.0,11.2,0.866,5,Pittsburgh Penguins,19831984


Team season stats are collected for every team currently in the NHL for years 1983 - 2020.

In [20]:
# Export data frame to project directory
nhl_df.to_csv('Data/1983_2020_NHL_Season_Data.csv', sep = ',', encoding = 'utf-8')

The data is exported to the project directory for further analysis.